In [108]:
import json
import urllib
import glob
import os
import time
import xml.etree.ElementTree as ET

In [109]:
user="aa.annaaniol@gmail.com" 
lpmn="any2txt|wcrft2" 
url="http://ws.clarin-pl.eu/nlprest2/base" 


In [110]:
potop_file = "../lab2/sources/potop.txt"
name_label = 'ign'
place_label = ''

In [111]:
with open (potop_file, "rb") as myfile:
    doc = myfile.read()
    upload_request = urllib.request.Request(url + '/upload/', doc, {'Content-Type': 'binary/octet-stream'})
    fileid = urllib.request.urlopen(upload_request).read().decode("utf-8") 

In [112]:
data = {'lpmn': lpmn, 'user': user, 'file':fileid}

doc = json.dumps(data)
binary_doc = doc.encode("utf-8")
# print(binary_doc)

In [113]:
task_request = urllib.request.Request(url + '/startTask/', binary_doc, {'Content-Type': 'application/json'})

In [114]:
taskid = urllib.request.urlopen(task_request).read()
taskid = taskid.decode("utf-8")

In [115]:
status_request = urllib.request.Request(url + '/getStatus/' + taskid)

In [118]:
# wait a moment for processing to be done and run

response = urllib.request.urlopen(status_request);
data = json.load(response)
print(data)

{'value': [{'name': 'file', 'fileID': '/requests/wcrft2/75eda6b4-8269-444c-8be3-eac4109fe166'}], 'status': 'DONE'}


In [119]:
file_id = data['value'][0]['fileID']
print(file_id)

/requests/wcrft2/75eda6b4-8269-444c-8be3-eac4109fe166


In [120]:
content_request = urllib.request.Request(url + '/download' + file_id)

In [121]:
content = urllib.request.urlopen(content_request).read()
content = content.decode("utf-8")
# print(content[:10000])

In [122]:
root = ET.fromstring(content)

In [123]:
def print_sentence(sentence):
    for tok in sentence:
        orth = tok.find('orth')
        if orth != None:
            print(orth.text, end=' ')

In [124]:
names = {}
places = []

In [125]:
for chunk in root:
    for sentence in chunk:
        for tok in sentence:
            if_name = False
            if_place = False
            for lex in tok:
                if lex != None:
                    ctag = lex.find('ctag')
                    if ctag != None:
                        roles = ctag.text.split(':')
                        if name_label in roles:
                            if_name = True
                        if place_label in roles:
                            if_place = True
                    base = lex.find('base')
                    if base != None and if_name:
                        if base.text in names:
                            names[base.text].append(sentence)
                        else:
                            names[base.text] = [sentence]             

In [126]:
len(names)

3070

In [127]:
def print_all_contexts(phrase):
    i = 1
    for sentence in names[phrase]:
        print('\n', i, 'occurrence:')
        print_sentence(sentence)
        i += 1

In [128]:
print_all_contexts('Billewicze')


 1 occurrence:
Gniazdo ich rodzinne , istniejące do dziś , zwało się także Billewicze , ale prócz nich posiadali wiele innych majętności i w okolicy Rosień , i dalej ku Krakinowu , wedle Laudy , Szoi , Niewiaży - aż hen , jeszcze za Poniewieżem . 
 2 occurrence:
Nie o sto mil Billewicze od Kiejdan , a z ciebie rara avispod moim dachem . 
 3 occurrence:
A cóż to , Billewicze - forteca ? 
 4 occurrence:
- Tymczasem pan pułkownik Billewicz - wielki to ród na Żmudzi ci Billewicze - wyzuł znikomą postawę i na lepszy świat się przeniósł , a mnie wioskę i wnuczkę zapisał . 
 5 occurrence:
Wszyscy Billewicze , którzy już są w polu , staną przy nas . 
 6 occurrence:
Dotrzymają Billewicze Radziwiłłom ! 
 7 occurrence:
- Tacyż to Billewicze tani , także to klejnot wasz szlachecki i starożytność rodu cenisz ? 
 8 occurrence:
A co panowie Sicińscy , nasi nieprzyjaciele , powiedzą , gdy Billewicze tak wyrosną ? 
 9 occurrence:
- Billewicze - mówił - krzyczą wniebogłosy na swą krzywdę i niewolę , a 

In [129]:
sorted_names = sorted(names.items(), key=lambda x: len(x[1]), reverse=True)

In [130]:
print('top 10 occurrrences of named entities:')
for name in sorted_names[:10]:
    print(name[0], len(name[1]))

top 10 occurrrences of named entities:
Babinicz 190
Skrzetuski 132
frame 97
NETOGRAFIA 97
Sakowicz 96
Kiejdan 73
Kiejdanach 71
Taurogach 70
Kuklinowski 63
Taurogów 56
